In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import plotly.express as px
from scipy.optimize import least_squares
from lmfit.models import LorentzianModel, GaussianModel, LinearModel
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
def get_df_from_dir(directory = "colocated_data_cr_1"):
    files = os.listdir(directory)
    df_list = []
    for file in tqdm(files):
        df_list.append(pd.read_csv(directory + "/" + file))
    return pd.concat(df_list)

def get_heatmap(arg = "wind"):
    fig = px.density_heatmap(df, y="mss", x=arg, color_continuous_scale=px.colors.sequential.Blackbody)
    fig.update_layout(
        xaxis_title= r"$\text{" + arg + "}$",
        yaxis_title= r"$\text{MSS}$",
        legend_title="Legend Title",
        font=dict(
            size=16,
        )
    )
    fig.show()


df = get_df_from_dir("colocated_data_cr_1")

In [ ]:
df = df[df['mss'] < 0.07]
df = df.dropna()
get_heatmap()

In [ ]:
gauss_model(8)

In [ ]:
model = LorentzianModel()
y = df['mss'].to_numpy()
x = df['wind'].to_numpy()
params = model.guess(y, x=x)
result = model.fit(y, params, x=x)

In [ ]:
result.plot_fit()
plt.show()

print(result.fit_report())

In [ ]:
def gauss_model(x):
    amplitude = 0.56965907 
    sigma = 5.90326384 
    center = 11.3767419  
    frac_1 = amplitude/(sigma*np.sqrt(2*np.pi))
    frac_2 = np.exp((-(x-center)**2)/(2*sigma**2))
    return frac_1/frac_2

In [ ]:
print(result)